In [1]:
import pandas as pd
from src.utils import MySQLAgent, read_config
from src.credit_invest import CreditInvest
import json

In [2]:
conn_path = ".env/connections.json"
configs = read_config(path=conn_path)
conn_configs = configs["CREDITREPORT"]['Crawler_mysql_conn_info']
credit_invest = CreditInvest(conn_configs=conn_configs)

In [3]:
company_id = '02853671'
query = f"""
                select business_accounting_no, fac_name, penalty_money, penalty_date, transgress_type, is_improve, penaltykind, paymentstate
                from epa_ems_p_46
                where Business_Accounting_No = {company_id} and penalty_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 YEAR)
            """
df_epa = credit_invest.read_table(query=query)
df_epa

,business_accounting_no,fac_name,penalty_money,penalty_date,transgress_type,is_improve,penaltykind,paymentstate
0,02853671,味榮食品工業股份有限公司,30000,2022-10-04,水污染,不須改善,罰鍰(非連續處罰),無分期已繳清
1,02853671,味榮食品工業股份有限公司,1200,2022-09-07,一般廢棄物,不須改善,罰鍰(非連續處罰),無分期已繳清
2,02853671,國森企業股份有限公司,1000,2024-03-29,移動污染,不須改善,罰鍰(非連續處罰),尚未繳款


In [12]:
# record count in terms of penaltykind
penaltykind_count = df_epa.groupby(['penaltykind']).size().reset_index(name="count")
# penalty_money in terms of penaltykind
df_epa['penalty_money'] = df_epa['penalty_money'].astype(int)
penaltykind_total_money = df_epa.groupby(['penaltykind']).agg(penaltykind_amount=('penalty_money','sum')).reset_index()
# is_improve in terms of penalty_kind
improve_state = df_epa.groupby(['penaltykind', 'is_improve']).size().reset_index()
# penalty_money of paymentstate = '尚未繳款'
penaltykind_unpay = df_epa[df_epa['paymentstate'] == '尚未繳款'].groupby(['penaltykind','paymentstate']).agg(penaltykind_amount=('penalty_money','sum')).reset_index()

In [10]:
epa_result = {
    'penaltykind_count': penaltykind_count.to_dict(orient='records'),
    'penaltykind_total_money': penaltykind_total_money.to_dict(orient='records'),
    'improve_state': improve_state.to_dict(orient='records'),
    'penaltykind_unpay': penaltykind_unpay.to_dict(orient='records')
}

In [13]:
penaltykind_count

,penaltykind,count
0,罰鍰(非連續處罰),3
